In [51]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import selenium
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore') 

##  웹사이트 크롤링

In [52]:
# 간단한 전처리 함수("- dc App" 문자열 제거 & url 제거 & 앞뒤 공백 제거) 
def prepro(comment):
    if "- dc App" in comment:
        comment = comment.replace("- dc App", "").strip()
    if "https://" in comment:
        comment = comment.split("https://")[0].strip()
    return comment

In [4]:
# example

# 크롬(Chrome) 브라우저 실행하기
browser = webdriver.Chrome('./chromedriver')
# 로깅(logging) 메시지 줄이기
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

url = "https://gall.dcinside.com/board/lists/?id=dcbest" #디시인사이드 실시간 베스트 갤러리
browser.get(url) # 웹사이트에 접속
time.sleep(3) # 3초간 정지

# 페이지 1
total_comment = []
for i in range(3, 52):
    browser.find_element_by_css_selector(f'#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({i}) > td.gall_tit.ub-word > a:nth-child(1)').click()
    time.sleep(4)
    full_html = browser.page_source
    soup = BeautifulSoup(full_html, 'html.parser')
    comments = soup.find_all('p', class_ = 'usertxt')
    for comment in comments:
        comment = comment.get_text().strip()
        total_comment.append(prepro(comment))
    time.sleep(4)
    browser.get(url)
    time.sleep(4)
browser.quit()

### > 페이지 number 별로 댓글 수집 
* 한 번 실행에 한 페이지씩 크롤링
* 한 게시판에 50개 이상의 게시글 → 페이지 넘기기까지 자동화해서 돌리니 접근 제한 or 기타 에러 등 발생 

In [53]:
browser = webdriver.Chrome('./chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

page_num = 42  # 페이지 number 지정
url = f"https://gall.dcinside.com/board/lists/?id=dcbest&page={page_num}&_dcbest=1"
browser.get(url)
time.sleep(3)

total_comment=[]
# 해당 페이지에 있는 모든 게시글을 방문해서 댓글 저장 (2번째 게시글 ~ 51번째 게시글)
for i in range(2, 52): 
    try:
        browser.find_element_by_css_selector(f'#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({i}) > td.gall_tit.ub-word > a:nth-child(1)').click()
    except: 
        continue
    time.sleep(3)
    full_html = browser.page_source
    soup = BeautifulSoup(full_html, 'html.parser')
    comments = soup.find_all('p', class_ = 'usertxt')
    for comment in comments:
        comment = comment.get_text().strip()
        total_comment.append(prepro(comment))
    time.sleep(2)
    browser.get(url)
    time.sleep(3)
browser.quit()

In [61]:
browser = webdriver.Chrome('./chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
page_num = 44
url = f"https://gall.dcinside.com/board/lists/?id=dcbest&page={page_num}&_dcbest=1"
browser.get(url)
time.sleep(3)

for i in range(2, 52):
    try:
        browser.find_element_by_css_selector(f'#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({i}) > td.gall_tit.ub-word > a:nth-child(1)').click()
    except: 
        continue
    time.sleep(3)
    full_html = browser.page_source
    soup = BeautifulSoup(full_html, 'html.parser')
    comments = soup.find_all('p', class_ = 'usertxt')
    for comment in comments:
        comment = comment.get_text().strip()
        total_comment.append(prepro(comment))
    time.sleep(2) 
    browser.get(url)
    time.sleep(3)
browser.quit()

In [62]:
len(total_comment)

215749

In [63]:
# 댓글 text 파일로 저장 (수시로 저장 요망)
with open('comment9.txt', 'w') as file:
    for comment in total_comment:
        file.write(comment + '\n')

## Text 파일 병합 및 정제

In [64]:
comment_list = ["comment", "comment1", "comment2", "comment3", "comment4", "comment5", "comment6", "comment7", "comment8", "comment9"]

total_comment = []
for comm_file in comment_list:
    with open(f"{comm_file}.txt", 'r') as file:
        comments = file.readlines()
        for commenet in comments:
            # 빈 문자열 & 한 글자 댓글 삭제
            if len(commenet) == 0 or len(commenet) == 1:
                continue
            else:
                total_comment.append(commenet.strip())
total_comment

['통피 인증박으니까 런치는거 존나웃기노 ㅋㅋㅋㅋㅋㅋ',
 '지는 지잡대 다니는데 실베에서도 죄다 지잡지잡거리니까 진짜 실베에 지잡대나 고졸만 있는줄 알았나보노 ㅋㅋ',
 'ㅋㅋ내가 성댄데 병신잡대새끼가 사칭하는게 역겨워서 하는 소린데?  저걸 인증이라 하는거임? Gls인증 못하고 애쓴다 너도',
 '223일단 넌 개잡대 앰생 확정이고ㅋㅋㅋ',
 '주문하신 금잔디 ㅋㅋ',
 '223 열등감 오지게 느끼나보네 ㅋㅋㅋ 같은 실베충인데 쟤는 명문대생이고 나는 앰생지잡이니 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '깔끔하게 인증박고 아닥시키는거 개멋잇노 ㅋㅋ',
 'Gls 말하니까 이 악물고 못 본 척하노..',
 '디시 잡대 빡대갈들 저게 인증인 줄 알고 빨아대는 것도 코미디ㅋㅋㅋㅋ',
 'ㄴ ㅋㅋ gls가 뭔진 아냐 ㅋㅋ',
 'sk통피게이 성균관대 인증 진짜로 올라와버리니 현실도피 on ㅋㅋㅋㅋㅋ 아니 그냥 그런갑다하면되는데 학벌컴플렉스 진짜 오지게 심하나보누',
 '통피친구야.. 이쯤되면 본인만 추해지니까 고마해라.. 형이 안타까워서 하는소리다',
 '너 성대라메? Gls인증하면 끝날 걸 뱅뱅 돌리는거 보면 잡대생인 거 티남',
 '아닌게 아니라 쟤가 무슨 설의나 하버드 프린스턴 다닌다고 한거도 아니고 그냥 성대다닌다 한건데 이것도 믿기힘들정도면 도대체 어떤 삶을 사는거노 ㄷㄷ;',
 '너같은 잡대생들은 그렇게 정신승리해야지ㅋㅋ',
 '학벌 조또 의미없니 어쩌구해도 진짜 여기 보면 역시 그래도 학벌이 받쳐주면 어디 한군데 믿는구석이 생겨서 그런가 ㅈㄴ 든든해보이긴함 긁을라해도 안긁히는게 이것만해도 솔직히 세상살아가는데 도움 마이 될 것 같음',
 'Gls인증하라니까 이 악물고 금잔디 타령하는 병신 성대 사칭 재밌노?',
 'ㄴ 나도 성대생인데 gls가 gold lawn square라고 금잔디 광장의 약자란다 잡대친구야 ㅋㅋㅋ 쟤는 금잔디 아는 거 보니까 성대생 맞긴 하노',
 '인증해달라해서 인중해주니까 자기가 몰라서 못 알아보노 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 씨발 뭐 꽁트찍냐 ㅋㅋ

In [65]:
len(total_comment)

419049

In [68]:
# 중복 댓글 삭제
unique = list(set(total_comment)) 

# 최종 병합 후 csv 파일로 변환
pd.DataFrame(unique).to_csv("./final_comments.csv")